# LagBox Measurements Converter

### Existing fields

#Mode:;3
#Device:;6312kw
#Button:;21
#minDelay:;100
#maxDelay:;10000
#iterations:;1000


### New fields to add
#author:;
#vendorId:;
#productId:;
#date:;
#bInterval:;
#deviceType:;
#email:;
#public:;
#notes:;
#EAN:;
#deviceSpeed:;

### Existing fields to rename
#Mode:;   ---->     #deviceType:;

## Imports

In [1]:
import os
import csv

## Constants

In [15]:
PATH_MEASUREMENTS = 'default_polling_test/'
ADDITIONAL_METADATA_FIELDS = ['#author:;',
                              '#vendorId:;',
                              '#productId:;',
                              '#date:;',
                              '#bInterval:;',
                              '#deviceType:;',
                              '#email:;',
                              '#public:;',
                              '#notes:Converted logfile from old format to new format -VM;',
                              '#EAN:;',
                              '#deviceSpeed:;']

DEVICE_NAMES ={"6312kw" : "Acer 6312-KW (PS/2)", "adns5700" : "Microsoft Optical Mouse 200", "apple1152" : "Apple A1152",
               "arduino" : "Arduino Micro","bt58" : "Logitech Premium Optical Wheel Mouse",
               "diamondback" : "Razor Diamondback","dragonrise_black" : "Speedlink Strike (Black)",
               "dragonrise_green" : "Speedlink Strike (Green)", "dualshock3_wired" : "Sony Dualshock 3",
               "dualshock4_wired" : "Sony Dualshock 4", "dualshock4_wireless" : "Sony Dualshock 4 wireless",
               "exxter" : "Exxter KE-200", "g15" : "Logitech G15", "g300" : "Logitech G300", "g5" : "Logitech G5",
               "g9" : "Logitech G9","g700_wired" : "Logitech G700", "g700_wireless" : "Logitech G700", "gembird" : "Gembird", 
               "intelli_new" : "Microsoft Intellimouse", "intelli_og" : "Microsoft Classic Intellimouse (PS/2)", 
               "k120" : "Logitech K120", "logilink0043" : "LogiLink ID0043", "nixdorf" : "Siemens Nixdorf (PS/2)", 
               "nvidiashield" : "nVidia Shield Gamepad", "nvidiashield_wireless" : "nVidia Shield Gamepad wireless", "px1800" : "PX-1800",
               "rapoo" : "Rapoo 6010B", "rival" : "Steel Series Rival Optical Mouse", "roccat" : "Roccat ISKU FX",
               "rx250" : "Logitech RX250", "sigma" : "Sigma", "sk9620" : "Acer SK-9620 (PS/2)", "sunplus" : "MS-TECH Laser Game Mouse",
               "tramani_wired" : "Tramani CT-P3000", "trust_oni_wireless" : "Trust Oni Wireless Micro Mouse",
               "unicomp" : "Unicomp Endura Pro", "wingman" : "Logitech Wingman", "xbox360_wired" : "XBox 360",
               "xbox360_wireless" : "XBox 360 wireless", "xboxone_wireless" : "XBox One Controller wireless"}

#TODO xbox360_wired polling rate
POLLING_RATES = {"6312kw" : 10, "adns5700" : 10, "apple1152" : 10, "arduino" : 1, 
                 "bt58" : 10, "diamondback" : 10, "dragonrise_black" : 10, "dragonrise_green" : 10, 
                 "dualshock3_wired" : 1, "dualshock4_wired" : 5, "dualshock4_wireless" : -1, "exxter" : 10, 
                 "g15" : 24, "g300" : 1, "g5" : 1, "g9": 10, "g700_wired" : 1, "g700_wireless" : -1, "gembird" : 10, 
                 "intelli_new" : 1, "intelli_og" : 10, "k120" : 10, "logilink0043" : 10, "nixdorf" : 10, 
                 "nvidiashield" : 1, "nvidiashield_wireless" : -1, "px1800" : 10, "rapoo" : -1, "rival" : 1, 
                 "roccat" : 1, "rx250" : 10, "sigma" : 10, "sk9620" : 10, "sunplus" : 10, 
                 "tramani_wired" : 10, "trust_oni_wireless" : 10, "unicomp" : 10, "wingman" : 10, 
                 "xbox360_wired" : 4, "xbox360_wireless" : 4, "xboxone_wireless" : -1}

## Helper functions


In [32]:
def get_device_name(file):
    file = file.replace('.csv', '')
    file_name_parts = file.split('_')
    device_name = file.replace(file_name_parts[0] + '_', '')
    device_name = device_name.replace('_' + file_name_parts[-1], '')
    device_name = device_name.replace('_' + file_name_parts[-2], '')
    print("Extracted device name:", device_name)
    return device_name


# Modifiy metadata
def modify_metadata(comment_lines, file, device_name, polling_rate):
    print("Modifing metadata", file)
    for entry in ADDITIONAL_METADATA_FIELDS:
        comment_lines.append(entry)
    
    print("Current: ", comment_lines)
    for i in range(len(comment_lines)):
        if "#Mode:;" in comment_lines[i]:
            comment_lines[i] = comment_lines[i].replace('Mode', 'deviceType')
        
        if "#bInterval:;" in comment_lines[i]:
            print("Appending polling rate")
            comment_lines[i] = comment_lines[i].replace('\n', '') + str(polling_rate) + 'ms' + '\n'
        
        if "#Device:;" in comment_lines[i]:
            comment_lines[i] = comment_lines[i].replace('\n', '').replace(file, '') + device_name + '\n'
    
    return comment_lines

# create modified version of file
def recreate_file(file, comment_lines, measurement_lines):
        print("Recreating file")
        print(comment_lines)
        
        with open(PATH_MEASUREMENTS + file, 'w') as f:
            writer = csv.writer(f)
            for comment in comment_lines:
                print(comment)
                writer.writerow([comment.replace('\n', '')])
            writer.writerow([])  # Add an empty line
            writer.writerow(['counter;latency;delayTime'])
            for measurement in measurement_lines:
                writer.writerow([measurement.replace('\n', '')])

## Read in csv

In [33]:
for file in os.listdir(PATH_MEASUREMENTS): 
    print(file)
    
    comment_lines = []  # All lines containing a comment (==> Metadata about the measurement)
    measurement_lines = []  # All lines containing actual measurement data
    
    device_name = get_device_name(file)
    present_device_name = device_name
    
    if (file.split('_')[-2] == '1ms'):
        polling_rate = 1
    else:
        polling_rate = 0
        try:
            polling_rate = POLLING_RATES[device_name]
        except:
            print('Could not get Polling rate')

    try:
        device_name = DEVICE_NAMES[device_name]
    except:
        print(device_name)
        
    print(device_name, polling_rate)
    
    
    print(file)
    current_file = open(PATH_MEASUREMENTS + file, 'r').readlines()  # Open the csv File
    for i in range(len(current_file)):
        if current_file[i][0] is '#':  # If row is a comment
            # print(line)
            comment_lines.append(current_file[i])
        elif current_file[i] == 'counter;latency;delayTime\n':  # If row is header of measurements
            measurement_lines = current_file[i + 1:len(current_file)]
            break

    comment_lines = modify_metadata(comment_lines, present_device_name, device_name, polling_rate)
    
    recreate_file(file, comment_lines, measurement_lines)
    
    print('\n')

AUTO_6312kw_0ms_1.csv
Extracted device name: 6312kw
Acer 6312-KW (PS/2) 10
AUTO_6312kw_0ms_1.csv
Modifing metadata 6312kw
Current:  ['#Mode:;3\n', '#Device:;6312kw\n', '#Button:;21\n', '#minDelay:;100\n', '#maxDelay:;10000\n', '#iterations:;1000\n', '#author:;', '#vendorId:;', '#productId:;', '#date:;', '#bInterval:;', '#deviceType:;', '#email:;', '#public:;', '#notes:Converted logfile from old format to new format -VM;', '#EAN:;', '#deviceSpeed:;']
Appending polling rate
Recreating file
['#deviceType:;3\n', '#Device:;Acer 6312-KW (PS/2)\n', '#Button:;21\n', '#minDelay:;100\n', '#maxDelay:;10000\n', '#iterations:;1000\n', '#author:;', '#vendorId:;', '#productId:;', '#date:;', '#bInterval:;10ms\n', '#deviceType:;', '#email:;', '#public:;', '#notes:Converted logfile from old format to new format -VM;', '#EAN:;', '#deviceSpeed:;']
#deviceType:;3

#Device:;Acer 6312-KW (PS/2)

#Button:;21

#minDelay:;100

#maxDelay:;10000

#iterations:;1000

#author:;
#vendorId:;
#productId:;
#date:;
#bInt